# COVID-19 Data (API).
### COVID-19 World Data.
[site](https://covid19api.com/) | [API info](https://documenter.getpostman.com/view/10808728/SzS8rjbc?version=latest)



In [0]:
# necessary imports for notebook.
import pandas as pd
import numpy as np
import requests
import json
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
''' COVID-19 : Countries Daily Summary Total'''
# COVID-19 Data.
'''https://covid19api.com/'''
'''https://documenter.getpostman.com/view/10808728/SzS8rjbc?version=latest'''
response = requests.get("https://api.covid19api.com/all")
# show the status code.
print(response.status_code)
covid_cs = response.json()
covid_countriessummary = pd.json_normalize(covid_cs)
# show the data frame shape.
print(covid_countriessummary.shape)
# show the data frame with headers.
covid_countriessummary.head()

200
(358530, 12)


,Country,CountryCode,Province,City,CityCode,Lat,Lon,Confirmed,Deaths,Recovered,Active,Date
0,Afghanistan,AF,,,,33.94,67.71,0,0,0,0,2020-01-22T00:00:00Z
1,Afghanistan,AF,,,,33.94,67.71,0,0,0,0,2020-01-23T00:00:00Z
2,Afghanistan,AF,,,,33.94,67.71,0,0,0,0,2020-01-24T00:00:00Z
3,Afghanistan,AF,,,,33.94,67.71,0,0,0,0,2020-01-25T00:00:00Z
4,Afghanistan,AF,,,,33.94,67.71,0,0,0,0,2020-01-26T00:00:00Z


In [0]:
# new data frame.
df = covid_countriessummary.copy()

In [0]:
# clean up the data.
df['Date'] = df['Date'].str[:10]
df = df.rename(columns={"CountryCode": "Code"})
df = df.drop(columns=['Province', 'City', 'CityCode', 'Active', 'Lat', 'Lon'])
df = df[['Date', 'Country', 'Code', 'Confirmed', 'Recovered', 'Deaths']]
df = df.groupby(['Date', 'Country', 'Code']).agg({'Confirmed':'sum', 'Deaths': 'sum', 'Recovered': 'sum'}).reset_index()
df = df[df['Country'] == 'United States of America']
df['ConDiff'] = df['Confirmed'].diff()
df['DeaDiff'] = df['Deaths'].diff()
df['RecDiff'] = df['Recovered'].diff()
df = df.fillna(0)
df['ConDiff'] = df['ConDiff'].astype(int)
df['DeaDiff'] = df['DeaDiff'].astype(int)
df['RecDiff'] = df['RecDiff'].astype(int)

In [11]:
print(df.shape)
df.head()

(86, 9)


,Date,Country,Code,Confirmed,Deaths,Recovered,ConDiff,DeaDiff,RecDiff
176,2020-01-22,United States of America,US,0,0,0,0,0,0
361,2020-01-23,United States of America,US,1,0,0,1,0,0
546,2020-01-24,United States of America,US,2,0,0,1,0,0
731,2020-01-25,United States of America,US,2,0,0,0,0,0
916,2020-01-26,United States of America,US,5,0,0,3,0,0


In [0]:
!pip install chart_studio

In [0]:
# set the browser for plotly.
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [0]:
# imports.
import pandas as pd
import plotly.graph_objs as go
import chart_studio.plotly as py
import plotly.offline
from plotly.offline import iplot, init_notebook_mode

In [0]:
# plotly chart set up.
configure_plotly_browser_state()
init_notebook_mode(connected=True)
trace0 = go.Scatter(
    x=df['Date'],
    y=list(df['ConDiff']),
    name="Daily New Cases",
    mode='lines+markers',
    line=dict(color="#5cb8e6", width = 3, dash= 'dot')
)
trace1 = go.Scatter(
    x=df['Date'],
    y=list(df['DeaDiff']),
    name="Daily Death Cases",
    mode='lines+markers',
    line=dict(color="#eb4034", width = 3, dash = 'dot'),
)
trace2 = go.Scatter(
    x=df['Date'],
    y=list(df['RecDiff']),
    name="Daily Recovered Cases",
    mode='lines+markers',
    line=dict(color="#42d65d", width = 3, dash = 'dot')
)
data = [trace0,trace1,trace2]
layout = go.Layout(title="USA COVID19 Numbers",
                  yaxis=dict(title="Daily Count", 
                              zeroline=False),
                  xaxis=dict(title="Date",
                              ),
                  margin=dict(l=20, r=20, t=75, b=20),
                  paper_bgcolor="whitesmoke",
                  autosize=False,
                  width=1000,
                  height=500                                    
                )                            
fig = go.Figure(data=data, layout=layout)
fig.show()

![PlotlyLineGraph](https://github.com/CVanchieri/CVanchieri.github.io/blob/master/img/PlotlyCOVID19LineGraphPost/5.png?raw=true)